In [1]:
# Imports
import ray
from ray.job_submission import JobSubmissionClient, JobStatus
import time
import subprocess

In [2]:
# Ray cluster information for connection
ray_head_ip = "kuberay-head-svc.kuberay.svc.cluster.local"
ray_head_port = 8265
ray_address = f"http://{ray_head_ip}:{ray_head_port}"
client = JobSubmissionClient(ray_address)

In [3]:
# Executors
def local_submission(n):
    print("Local submission:")
    command = ["python", "ray-fibonacci-example.py", n]
    subprocess.run(command)

def ray_submission(n, c):
    print("Ray submission:")
    job_id = c.submit_job(
        entrypoint=f"python ray-fibonacci-example.py {n}",
        runtime_env={
            "working_dir": "./", 
        },
        entrypoint_num_cpus=3
    )

    # Waiting for Ray to finish the job and print the result
    while True:
        status = c.get_job_status(job_id)
        if status in [ray.job_submission.JobStatus.RUNNING, ray.job_submission.JobStatus.PENDING]:
            time.sleep(5)
        else:
            break
    print(c.get_job_logs(job_id))

In [4]:
# Running Fibonacci with 1k squence
sequence = "1000"

# Local
local_submission(sequence)

# Ray
ray_submission(sequence, client)

2024-03-13 19:04:29,411	INFO dashboard_sdk.py:338 -- Uploading package gcs://_ray_pkg_3020e9c8b7845023.zip.
2024-03-13 19:04:29,413	INFO packaging.py:530 -- Creating a file package for local directory './'.


Local submission:
Computation time for 1000 sequence is 0.01712322235107422
Ray submission:
Computation time for 1000 sequence is 0.03003239631652832



In [5]:
# Running Fibonacci with 50k squence
sequence = "50000"

# Local
local_submission(sequence)

# Ray
ray_submission(sequence, client)

Local submission:


2024-03-13 19:05:08,937	INFO dashboard_sdk.py:385 -- Package gcs://_ray_pkg_3020e9c8b7845023.zip already exists, skipping upload.


Computation time for 50000 sequence is 9.324785470962524
Ray submission:
Computation time for 50000 sequence is 4.688970327377319



In [46]:
# Disconnect from the Ray cluster
ray.shutdown()